# FP Growth

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("fpgrowth").getOrCreate()

24/04/03 16:51:10 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlo1)
24/04/03 16:51:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 16:51:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import split

In [3]:
transactions = spark.read.text("../0_data/Transacoes.txt").select(split("value", "\s+").alias("transactions"))
transactions.show(truncate=False)

+---------------------------+
|transactions               |
+---------------------------+
|[Chocolate, Pizza, Sorvete]|
|[Pizza, Sorvete]           |
|[Chocolate, Pizza]         |
|[Chocolate, Pizza, Sorvete]|
|[Chocolate, Pizza]         |
|[Pizza]                    |
+---------------------------+



In [4]:
fp = FPGrowth(itemsCol="transactions", minSupport=0.5, minConfidence=0.5)
model = fp.fit(transactions)
model.freqItemsets.show()

+------------------+----+
|             items|freq|
+------------------+----+
|         [Sorvete]|   3|
|  [Sorvete, Pizza]|   3|
|       [Chocolate]|   4|
|[Chocolate, Pizza]|   4|
|           [Pizza]|   6|
+------------------+----+



In [5]:
model.associationRules.show()

+-----------+-----------+------------------+----+------------------+
| antecedent| consequent|        confidence|lift|           support|
+-----------+-----------+------------------+----+------------------+
|  [Sorvete]|    [Pizza]|               1.0| 1.0|               0.5|
|[Chocolate]|    [Pizza]|               1.0| 1.0|0.6666666666666666|
|    [Pizza]|  [Sorvete]|               0.5| 1.0|               0.5|
|    [Pizza]|[Chocolate]|0.6666666666666666| 1.0|0.6666666666666666|
+-----------+-----------+------------------+----+------------------+



In [6]:
model.transform(transactions).show(truncate=False)

+---------------------------+--------------------+
|transactions               |prediction          |
+---------------------------+--------------------+
|[Chocolate, Pizza, Sorvete]|[]                  |
|[Pizza, Sorvete]           |[Chocolate]         |
|[Chocolate, Pizza]         |[Sorvete]           |
|[Chocolate, Pizza, Sorvete]|[]                  |
|[Chocolate, Pizza]         |[Sorvete]           |
|[Pizza]                    |[Sorvete, Chocolate]|
+---------------------------+--------------------+

